In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import TimeSeriesSplit
import optuna
import os

In [9]:

# Funkce pro optimalizaci pomocí Optuny
def objective(trial):
    interval = trial.suggest_int('interval', 45, 80)
    split_count = trial.suggest_int('split_count', 2, 10)
    epochs_count = trial.suggest_int('epochs_count', 1, 10)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    lstm1_units = trial.suggest_int('lstm1_units', 5, 150)
    lstm2_units = trial.suggest_int('lstm2_units', 5, 150)

    df = pd.read_csv('sorted_test_data.csv', parse_dates=['Datum'])
    df['Timestamp'] = df['Datum'].astype('int64')
    data = df[['Den_v_tydnu', 'Porucha','Timestamp']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(data)-interval):
        X.append(scaled_data[i:i+interval])
        y.append(1 if any(scaled_data[i+interval:i+interval+7, 1] == 1) else 0)
    X, y = np.array(X), np.array(y)

    tscv = TimeSeriesSplit(n_splits=split_count)
    X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []
    for train_index, test_index in tscv.split(X):
        X_train_list.append(X[train_index])
        X_test_list.append(X[test_index])
        y_train_list.append(y[train_index])
        y_test_list.append(y[test_index])

    model = Sequential()
    model.add(LSTM(units=lstm1_units, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(units=lstm2_units, return_sequences=False))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    train_loss_history = []
    val_loss_history = []

    for i in range(len(X_train_list)):
        X_train_fold, y_train_fold = X_train_list[i], y_train_list[i]
        X_test_fold, y_test_fold = X_test_list[i], y_test_list[i]
        history = model.fit(X_train_fold, y_train_fold, epochs=epochs_count, batch_size=batch_size, validation_data=(X_test_fold, y_test_fold))
        train_loss_history.append(history.history['loss'][0])
        val_loss_history.append(history.history['val_loss'][0])

    # Uložení výsledků po každé iteraci
    current_results = pd.DataFrame({
        'interval': [interval],
        'split_count': [split_count],
        'epochs_count': [epochs_count],
        'batch': [batch_size],
        'lstm1_units': [lstm1_units],
        'lstm2_units': [lstm2_units],
        'val_loss': [np.mean(val_loss_history)]
    })

    csv_path = 'optimization_results.csv'
    if os.path.exists(csv_path):
        current_results.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        current_results.to_csv(csv_path, index=False)

    return np.mean(val_loss_history)

In [10]:
# Název studie
study_name = 'lstm_optimization'
# Cesta k souboru s databází
storage_name = 'sqlite:///{}.db'.format(study_name)

# Zkontroluj, zda soubor studie již existuje
if os.path.exists(study_name + ".db"):
    print('Existující studie načtena')
    # Pokud ano, načti existující studii
    study = optuna.load_study(study_name=study_name, storage=storage_name)
else:
    # Pokud ne, vytvoř novou studii
    print('Vytvořena nová studie')
    study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)

# Spusť optimalizaci
study.optimize(objective, n_trials=1000, timeout=60*60*4)  # Timeout v sekundách


Existující studie načtena
Epoch 1/8
8/8 [==============================] - 6s 248ms/step - loss: 0.6693 - val_loss: 0.6527
Epoch 2/8
8/8 [==============================] - 1s 104ms/step - loss: 0.6459 - val_loss: 0.6467
Epoch 3/8
8/8 [==============================] - 1s 104ms/step - loss: 0.6491 - val_loss: 0.6476
Epoch 4/8
8/8 [==============================] - 1s 117ms/step - loss: 0.6461 - val_loss: 0.6470
Epoch 5/8
8/8 [==============================] - 1s 113ms/step - loss: 0.6635 - val_loss: 0.6636
Epoch 6/8
8/8 [==============================] - 1s 118ms/step - loss: 0.6444 - val_loss: 0.6484
Epoch 7/8
8/8 [==============================] - 1s 116ms/step - loss: 0.6535 - val_loss: 0.6518
Epoch 8/8
8/8 [==============================] - 1s 115ms/step - loss: 0.6523 - val_loss: 0.6484
Epoch 1/8
15/15 [==============================] - 2s 105ms/step - loss: 0.6474 - val_loss: 0.6055
Epoch 2/8
15/15 [==============================] - 2s 103ms/step - loss: 0.6465 - val_loss: 0.6061


[I 2023-11-15 14:51:42,594] Trial 191 finished with value: 0.5929744839668274 and parameters: {'interval': 56, 'split_count': 3, 'epochs_count': 8, 'batch_size': 32, 'lstm1_units': 44, 'lstm2_units': 102}. Best is trial 137 with value: 0.55778240164121.


Epoch 1/9
5/5 [==============================] - 7s 510ms/step - loss: 0.6776 - val_loss: 0.6318
Epoch 2/9
5/5 [==============================] - 1s 252ms/step - loss: 0.6609 - val_loss: 0.6254
Epoch 3/9
5/5 [==============================] - 1s 243ms/step - loss: 0.6631 - val_loss: 0.6249
Epoch 4/9
5/5 [==============================] - 1s 240ms/step - loss: 0.6606 - val_loss: 0.6284
Epoch 5/9
5/5 [==============================] - 1s 243ms/step - loss: 0.6577 - val_loss: 0.6195
Epoch 6/9
5/5 [==============================] - 1s 237ms/step - loss: 0.6511 - val_loss: 0.6225
Epoch 7/9
5/5 [==============================] - 1s 250ms/step - loss: 0.6397 - val_loss: 0.6238
Epoch 8/9
5/5 [==============================] - 1s 245ms/step - loss: 0.6241 - val_loss: 0.5837
Epoch 9/9
5/5 [==============================] - 1s 260ms/step - loss: 0.6424 - val_loss: 0.6126
Epoch 1/9
10/10 [==============================] - 2s 218ms/step - loss: 0.6059 - val_loss: 0.6246
Epoch 2/9
10/10 [===========

[I 2023-11-15 14:52:18,582] Trial 192 finished with value: 0.6282033622264862 and parameters: {'interval': 74, 'split_count': 2, 'epochs_count': 9, 'batch_size': 64, 'lstm1_units': 71, 'lstm2_units': 86}. Best is trial 137 with value: 0.55778240164121.


Epoch 1/7
6/6 [==============================] - 6s 320ms/step - loss: 0.6718 - val_loss: 0.6559
Epoch 2/7
6/6 [==============================] - 1s 120ms/step - loss: 0.6989 - val_loss: 0.6479
Epoch 3/7
6/6 [==============================] - 1s 123ms/step - loss: 0.6701 - val_loss: 0.6654
Epoch 4/7
6/6 [==============================] - 1s 140ms/step - loss: 0.6776 - val_loss: 0.6772
Epoch 5/7
6/6 [==============================] - 1s 122ms/step - loss: 0.6842 - val_loss: 0.6811
Epoch 6/7
6/6 [==============================] - 1s 122ms/step - loss: 0.6840 - val_loss: 0.6753
Epoch 7/7
6/6 [==============================] - 1s 121ms/step - loss: 0.6801 - val_loss: 0.6670
Epoch 1/7
10/10 [==============================] - 1s 126ms/step - loss: 0.6700 - val_loss: 0.6387
Epoch 2/7
10/10 [==============================] - 1s 131ms/step - loss: 0.6614 - val_loss: 0.6156
Epoch 3/7
10/10 [==============================] - 1s 131ms/step - loss: 0.6626 - val_loss: 0.6223
Epoch 4/7
10/10 [=======

...

[I 2023-11-15 18:48:28,791] Trial 416 finished with value: 0.5834087133407593 and parameters: {'interval': 68, 'split_count': 3, 'epochs_count': 9, 'batch_size': 32, 'lstm1_units': 89, 'lstm2_units': 19}. Best is trial 326 with value: 0.5558398962020874.


Epoch 1/8
6/6 [==============================] - 12s 531ms/step - loss: 0.6797 - val_loss: 0.6472
Epoch 2/8
6/6 [==============================] - 1s 166ms/step - loss: 0.6568 - val_loss: 0.6470
Epoch 3/8
6/6 [==============================] - 1s 143ms/step - loss: 0.6550 - val_loss: 0.6440
Epoch 4/8
6/6 [==============================] - 1s 144ms/step - loss: 0.6522 - val_loss: 0.6420
Epoch 5/8
6/6 [==============================] - 1s 148ms/step - loss: 0.6517 - val_loss: 0.6391
Epoch 6/8
6/6 [==============================] - 1s 148ms/step - loss: 0.6484 - val_loss: 0.6720
Epoch 7/8
6/6 [==============================] - 1s 148ms/step - loss: 0.6435 - val_loss: 0.6466
Epoch 8/8
6/6 [==============================] - 1s 157ms/step - loss: 0.6374 - val_loss: 0.6634
Epoch 1/8
12/12 [==============================] - 2s 142ms/step - loss: 0.6483 - val_loss: 0.6281
Epoch 2/8
12/12 [==============================] - 1s 126ms/step - loss: 0.6351 - val_loss: 0.6598
Epoch 3/8
12/12 [========

[I 2023-11-15 18:49:41,312] Trial 417 finished with value: 0.5740383267402649 and parameters: {'interval': 66, 'split_count': 4, 'epochs_count': 8, 'batch_size': 32, 'lstm1_units': 95, 'lstm2_units': 23}. Best is trial 326 with value: 0.5558398962020874.


Epoch 1/9
4/4 [==============================] - 11s 820ms/step - loss: 0.6918 - val_loss: 0.6398
Epoch 2/9
4/4 [==============================] - 0s 105ms/step - loss: 0.6781 - val_loss: 0.6317
Epoch 3/9
4/4 [==============================] - 0s 104ms/step - loss: 0.6757 - val_loss: 0.6206
Epoch 4/9
4/4 [==============================] - 0s 103ms/step - loss: 0.6744 - val_loss: 0.6125
Epoch 5/9
4/4 [==============================] - 0s 107ms/step - loss: 0.6717 - val_loss: 0.6249
Epoch 6/9
4/4 [==============================] - 0s 97ms/step - loss: 0.6710 - val_loss: 0.6310
Epoch 7/9
4/4 [==============================] - 0s 86ms/step - loss: 0.6695 - val_loss: 0.6289
Epoch 8/9
4/4 [==============================] - 0s 108ms/step - loss: 0.6674 - val_loss: 0.6259
Epoch 9/9
4/4 [==============================] - 0s 102ms/step - loss: 0.6648 - val_loss: 0.6092
Epoch 1/9
7/7 [==============================] - 1s 88ms/step - loss: 0.6423 - val_loss: 0.6840
Epoch 2/9
7/7 [=================

[I 2023-11-15 18:51:33,405] Trial 418 finished with value: 0.6486452288097806 and parameters: {'interval': 52, 'split_count': 9, 'epochs_count': 9, 'batch_size': 32, 'lstm1_units': 77, 'lstm2_units': 17}. Best is trial 326 with value: 0.5558398962020874.


In [11]:
# Výsledky optimalizace
print('Best trial:')
trial = study.best_trial
print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


Best trial:
Value:  0.5558398962020874
Params: 
    interval: 72
    split_count: 3
    epochs_count: 10
    batch_size: 64
    lstm1_units: 82
    lstm2_units: 100
